In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from print2log import redirect_to_logger
from ftp_utils import create_connect
from typing import Optional
import mylogger
import logging
from pathlib import Path
from ftplib import FTP, error_perm
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

logger = logging.getLogger()

In [ ]:
FTP_HOST = "localhost"
FTP_USER = "ITuser"
FTP_PSWD = "1234"

TARGET_PATH = "./result/work2"
TARGET_TYPE = "JPG"

TIMEZONE = 'Asia/Taipei'

In [ ]:
def download_dir(ftp: FTP, source: str, target: str | Path, target_type: str = TARGET_TYPE):
    def match_type(attr: dict, filename: str, target_type: str):
        return attr["type"] == "file" and filename.upper().endswith(f".{target_type}")

    lst_file = [filename for filename, attr in ftp.mlsd(source) if match_type(attr, filename, target_type)]
    logger.info(f"Get File List from [{source}]: {lst_file}")

    local_path = Path(target) / source
    local_path.mkdir(exist_ok=True, parents=True)
    for filename in lst_file:
        file_path = local_path / filename
        with open(file_path, "wb") as fp:
            ftp.retrbinary(f"RETR {source}/{filename}", fp.write)


def get_download_dir(lst_dir: list[str], start_time: Optional[str], end_time: Optional[str]):
    def dir_in_time_range(dir_name: str, start_time: str, end_time: str):
        DIR_TIME_FORMAT = "%Y%m%d%H%M%S_CDSC"
        TIME_FORMAT = "%Y/%m/%d %H:%M:%S"
        dir_time = datetime.strptime(dir_name, DIR_TIME_FORMAT)
        return start_time <= dir_time.strftime(TIME_FORMAT) <= end_time

    if isinstance(start_time, str) and isinstance(end_time, str):
        logger.info(f"Search Folder between [{start_time}] ~ [{end_time}]")
        lst_download_dir = [dir for dir in lst_dir if dir_in_time_range(dir, start_time, end_time)]
    else:
        lst_download_dir = [max(lst_dir)]
    return lst_download_dir


def get_dates_in_range(start_time: Optional[str] = None, end_time: Optional[str] = None):
    DATE_TIME_FORMAT = "%Y/%m/%d %H:%M:%S"
    DATE_FORMAT = "%Y/%m/%d"
    date_list = []

    if isinstance(start_time, str) and isinstance(end_time, str):
        try:
            st = datetime.strptime(start_time, DATE_TIME_FORMAT)
            et = datetime.strptime(end_time, DATE_TIME_FORMAT)
        except ValueError as e:
            logger.error(f"Parsing Time Failed: {e}", exc_info=True)
            return date_list

        cur_date = st.date()
        while cur_date <= et.date():
            date_list.append(cur_date.strftime(DATE_FORMAT))
            cur_date += timedelta(days=1)

        return date_list
    else:
        cur_date = datetime.now(ZoneInfo(TIMEZONE))
        return [cur_date.strftime(DATE_FORMAT)]


@redirect_to_logger(msg="FTP")
def get_data_from_scc(
    host: str,
    user: str,
    pswd: str,
    toolId: str,
    local_path: str | Path,
    start_time: Optional[str] = None,
    end_time: Optional[str] = None,
):
    root = f"/CDSC/{toolId}/"

    lst_date = get_dates_in_range(start_time, end_time)
    logger.info(f"Walking Folders: {lst_date}")

    with create_connect(host, user, pswd) as ftp:
        for date in lst_date:
            target_folder = root + date
            try:
                ftp.cwd(target_folder)
                logger.info(f"Change Folder to [{target_folder}]")
            except error_perm as e:
                logger.error(f"Change Folder to [{target_folder}] Failed: {e}")
                continue

            lst_dir = [name for name, attr in ftp.mlsd() if attr["type"] == "dir"]
            logger.info(f"Found Folder in [{target_folder}]: {lst_dir}")

            lst_download_dir = get_download_dir(lst_dir, start_time, end_time)
            logger.info(f"Download Folder List: {lst_download_dir}")
            for dir in lst_download_dir:
                download_dir(ftp, dir, local_path)


get_data_from_scc(
    host=FTP_HOST,
    user=FTP_USER,
    pswd=FTP_PSWD,
    toolId="TOOL1",
    local_path=TARGET_PATH,
    start_time="2025/11/01 18:20:35",
    end_time="2025/12/15 18:20:40",
)